# 7. Rerun everything with yields created from five year rolling mean
## Forecasting Crop Yields on a national scale (FACYnation)

### by Raphael Shirley (University of Sussex)

In this notebook we take the previous model and investigate increases in temperature.

## Investigating impact of temperature increases

In this notebook we take the two dimensional Gaussian fitted to the regional data and check for the impact of temperature increases which should depend on where current temperatures lie with respect to the peak yield response.









## Rerun model from notebook 3
First lets generate the same set of samples from the posterior.

In [ ]:
import pandas as pd
import pylab as plt
import numpy as np
import seaborn as sns
import pystan

import time

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

from xidplus.stan_fit import stan_utility

%matplotlib inline


## Compute the productivity trend and find a final baseline

In [ ]:
yields_for_comp = pd.read_table('./Crop_data_files/Maize_yield_obs_timeseries.csv', sep=',')
anoms_for_comp = pd.read_table('./Crop_data_files/Maize_median_yield_anoms.csv')

In [ ]:
states = ['Indiana', 'Illinois', 'Ohio', 'Nebraska', 'Iowa', 'Minnesota']
state = 'Nebraska'

years = np.arange(1960, 2008)
anoms_by_year = [anoms_for_comp[anoms_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
yields_by_year = [yields_for_comp[yields_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]

In [ ]:
def moving_mean(temp, scale=5): # Moving average by numpy convolution
    #print(temp)
    temp_padded = np.pad(temp, (scale//2, scale-1-scale//2), mode='edge')
    #print(temp_padded)
    smoothed=np.convolve(temp_padded, np.ones((scale,))/scale, mode='valid') 
    return smoothed

mv = moving_mean(yields_by_year)
print("Original length: {}, new length: {}".format(len(yields_by_year), len(mv)))
print(mv)

In [ ]:
for state in states:
    anoms_by_year = [anoms_for_comp[anoms_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
    yields_by_year = [yields_for_comp[yields_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
    mean_anoms_by_year = moving_mean(yields_by_year)

In [ ]:
fig, ax = plt.subplots()


for state in states:
    anoms_by_year = [anoms_for_comp[anoms_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
    yields_by_year = [yields_for_comp[yields_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
    mean_anoms_by_year = yields_by_year - moving_mean(yields_by_year)
    
    plt.scatter(years, mean_anoms_by_year, s = 15.0, alpha=0.5, c ='g')
    plt.scatter(years, anoms_by_year, s = 15.0, alpha=0.5, c ='b')
    plt.scatter(years, yields_by_year, s = 15.0, alpha=0.5, c ='r')
    
plt.scatter([-1], [0], s = 2.0, c ='r', label = 'actual values')
plt.scatter([-1], [0], s = 2.0, c ='b', label = '5 year median anomalies')
plt.scatter([-1], [0], s = 2.0, c ='g', label = '5 year mean anomalies')

ax.set_xlabel('year')
ax.set_ylabel('Yield [tonnes ha$^{-1}$]')
ax.set_xlim(1960, 2007)
plt.legend()

plt.savefig("./figs/yields_vs_mean_anoms_noline.png")
plt.savefig("./figs/yields_vs_mean_anoms_noline.pdf")

In [ ]:
all_anoms = np.array([])
all_yields = np.array([])
all_years = np.array([])
for state in states:
    anoms_by_year = [anoms_for_comp[anoms_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
    yields_by_year = [yields_for_comp[yields_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)][str(year)].iloc[0] 
                 for year in years]
    mean_anoms_by_year = yields_by_year - moving_mean(yields_by_year)
    all_yields = np.hstack([all_yields, yields_by_year])
    all_anoms = np.hstack([all_anoms, mean_anoms_by_year])
    all_years = np.hstack([all_years, years])

In [ ]:
fig, ax = plt.subplots()


import scipy
slope, intercept, rval, pval, sterr = scipy.stats.linregress(all_years, all_yields)

x = np.array([1950., 2020.])
y = slope * x + intercept
plt.plot(x, y, c ='r')
   
plt.scatter( all_years,  all_yields, s = 15.0, alpha=0.5, c ='r', label = 'Actual values')
plt.scatter( all_years,  all_anoms, s = 15.0, alpha=0.5, c ='b', label = 'Anomalies')

    
#plt.scatter([-1], [0], s = 5.0, c ='b', label = 'anomalies')
#plt.scatter([-1], [0], s = 5.0, c ='r', label = 'actual values')


ax.set_xlabel('Year')
ax.set_ylabel('Yield [tonnes ha$^{-1}$]')
ax.set_xlim(1960, 2007)
ax.set_ylim(-4, 12)
plt.legend()

plt.savefig("./figs/yields_vs_mean_anoms.png")
plt.savefig("./figs/yields_vs_mean_anoms.pdf")

In [ ]:
mean_anoms = yields_for_comp.copy()

In [ ]:
all_anoms[5]

In [ ]:
all_regions = mean_anoms['Region']
all_anoms = np.array([])
all_yields = np.array([])
all_years = np.array([])
for m, state in enumerate(all_regions):
    print(state)

    yields_by_year = [yields_for_comp[yields_for_comp['Region'] == state][str(year)].iloc[0] 
                 for year in years]
    mean_anoms_by_year = yields_by_year - moving_mean(yields_by_year)
    all_yields = np.hstack([all_yields, yields_by_year])
    all_anoms = np.hstack([all_anoms, mean_anoms_by_year])
    all_years = np.hstack([all_years, years])
    
    for n,year in enumerate(years):
        mean_anoms.loc[m, str(year)] = all_anoms[n]

In [ ]:
mean_anoms

## Write rolling five year mean anomolies to csv for use throughout

In [ ]:
mean_anoms.to_csv('./Crop_data_files/Maize_mean_yield_anoms.csv')

## Rerun full notebook as notebook 6 but with five year rolling mean anomalies

In [ ]:
print('predicted value in 2019 is {}'.format(slope * 2019 + intercept) )
print('latest value in 2007 is {}'.format(slope * 2007 + intercept) )

In [ ]:
all_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = all_months[3:9]

In [ ]:
def take_mean_anom(table):
    means_by_year = []
    for year in np.arange(1980, 2008):
        means_by_year.append(np.mean(
            [table[str(month)][table['Unnamed: 0'] ==  year].iloc[0]  for month in months ]
        ) )
    return np.array(means_by_year)
    
precip_mean_anoms = np.array([])
temp_mean_anoms   = np.array([])
yields_from_1980  = np.array([])

for state in states:
    precip_month_anoms = pd.read_table(
        'Crop_data_files/maize_met_anoms/Maize_Spring_USA_{}_precip_anom_real.csv'.format(state))
    temp_month_anoms = pd.read_table(
        'Crop_data_files/maize_met_anoms/Maize_Spring_USA_{}_temp_anom_real.csv'.format(state))

    precip_mean_anoms = np.hstack([precip_mean_anoms,
                                   take_mean_anom(precip_month_anoms)])
    temp_mean_anoms = np.hstack([temp_mean_anoms,
                                 take_mean_anom(temp_month_anoms)])
    yields_from_1980 = np.hstack([ yields_from_1980,
                                  ([yields_for_comp[
                                      yields_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)
                                  ][str(year)].iloc[0] 
                 for year in np.arange(1980, 2008)]) 
                                 ])
    


In [ ]:
# Scatter plot of the sources.
fig, axis = plt.subplots()


im = axis.scatter(temp_mean_anoms, 
                  precip_mean_anoms, 
                  c=yields_from_1980,  
                  cmap="viridis",
                  s=50.0,
                 alpha = 1.)#vmin=0.0, vmax=360.,
axis.set_xlabel("Average monthly temperature anomaly [K]")
axis.set_ylabel("Average monthly precipitation anomaly [mm]")
#axis.axis('equal')

cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# Optionally add a colorbar
#cax, _ = mpl.colorbar.make_axes(ax)
#cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap)


plt.savefig('./figs/real_data_scatter.pdf', bbox_inches='tight')
plt.savefig('./figs/real_data_scatter.png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()


test = np.full([10, 10], np.nan)

ts = np.linspace(-2,2, 10)
ps = np.linspace(-60,60, 10)

t_width = np.absolute(ts[1] - ts[0])
p_width = np.absolute(ps[1] - ps[0])

X,Y = meshgrid(ts, ps) # grid of point

def z_func(X,Y):
    in_bin = temp_mean_anoms > (X - t_width/2)
    in_bin = np.bitwise_and(in_bin , temp_mean_anoms < (X + t_width/2) )
    in_bin = np.bitwise_and(in_bin , precip_mean_anoms > (Y - p_width/2) )
    in_bin = np.bitwise_and(in_bin , precip_mean_anoms < (Y + p_width/2) )
    yields_temp = in_bin*yields_from_1980
    yields_temp[np.isclose(yields_temp , 0)] = np.nan
    return np.nanmean(yields_temp)


z_func = np.vectorize(z_func)
#Lets use the median

Z = z_func(X, Y)

im = imshow(np.flip(Z, axis=0),cmap="viridis", 
            extent=[ts[0],ts[-1] ,ps[0], ps[-1]],
            aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
cbar = colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

ax.set_xlabel('Monthly mean $T_{inc}$ [K]')
ax.set_ylabel('Monthly mean $P_{inc}$ [mm]')

plt.savefig('./figs/real_data_hist.pdf', bbox_inches='tight')
plt.savefig('./figs/real_data_hist.png', bbox_inches='tight')

In [ ]:

for state in states:
    precip_month_anoms = pd.read_table(
        'Crop_data_files/maize_met_anoms/Maize_Spring_USA_{}_precip_anom_real.csv'.format(state))
    temp_month_anoms = pd.read_table(
        'Crop_data_files/maize_met_anoms/Maize_Spring_USA_{}_temp_anom_real.csv'.format(state))
    
    yields_by_year = [yields_for_comp[yields_for_comp['Region'] == 'Maize_Spring_USA_{}'.format(state)
                                     ][str(year)].iloc[0] 
                 for year in years]
    
    #temp_mean =  np.nanmean(np.array(temp_month_anoms[all_months]))
    temp_variance = np.nanstd(np.array(temp_month_anoms[all_months])) 
    #precip_mean =  np.nanmean(np.array(precip_month_anoms[all_months]))
    precip_variance = np.nanstd(np.array(precip_month_anoms[all_months])) 
    yield_variance = np.nanstd(np.array(yields_by_year)) 
    
    print("{} & {} & {} & {} \\\\ ".format(state, 
                                 #round(temp_mean, 2), 
                                 round(temp_variance, 2), 
                                 #round(precip_mean, 2),
                                 round(precip_variance, 2),
                                 round(yield_variance, 2)
                                     )
         )

In [ ]:
precip_month_anoms = pd.read_table(
        'Crop_data_files/maize_met_anoms/Maize_Spring_USA_{}_precip_anom_real.csv'.format(state))
np.array(precip_month_anoms[all_months])

## Rerun the full models

In [ ]:
# Read in climate temperatures
clim_temp_maize=pd.read_table('./Crop_data_files/clim_file/temp_climatology_Maize.csv')
clim_temp_maize.rename(columns = {'Unnamed: 0':'Crop_season_location'}, inplace = True)
# Read in climate precipitation
clim_precip_maize=pd.read_table('./Crop_data_files/clim_file/precip_climatology_Maize.csv')
clim_precip_maize.rename(columns = {'Unnamed: 0':'Crop_season_location'}, inplace = True)
# Read in Yields
yields=pd.read_table('./Crop_data_files/Maize_mean_yield_anoms.csv')

In [ ]:
states=['Indiana','Illinois', 'Ohio','Nebraska', 'Iowa','Minnesota']

In [ ]:
#Read in and add back mean temperature to get real temperature values
temp_states=[]
for i,s in enumerate(states):
    maize_temp=pd.read_table('./Crop_data_files/maize_met_anoms/Maize_Spring_USA_'
                             +s+'_temp_anom_real.csv')
    maize_temp.rename(columns = {'Unnamed: 0':'Year'}, inplace = True)
    tmp=maize_temp.iloc[:,1:].add(clim_temp_maize[
        clim_temp_maize['Crop_season_location']== 'Maize_Spring_USA_'+states[0]
                                                 ].iloc[0,1:,])
    temp_states.append(tmp)
temp_states=pd.concat(temp_states,keys=states)

#Read in and add back mean precipitation to get real precipitation values
precip_states=[]
for i,s in enumerate(states):
    maize_precip=pd.read_table('./Crop_data_files/maize_met_anoms/Maize_Spring_USA_'
                               +s+'_precip_anom_real.csv')
    maize_precip.rename(columns = {'Unnamed: 0':'Year'}, inplace = True)
    tmp=maize_precip.iloc[:,1:].add(clim_precip_maize[
        clim_precip_maize['Crop_season_location']== 'Maize_Spring_USA_'+states[0]
                                                     ].iloc[0,1:,])
    precip_states.append(tmp)
precip_states=pd.concat(precip_states,keys=states)

In [ ]:
n_years=np.array(yields[yields['Region']=='Maize_Spring_USA_Indiana'].iloc[0,22:]).size
data2={
    'n_regions':len(states),
    'n_years':n_years,
    'd_temp':np.array(temp_states.iloc[:,3:9]).reshape(
                     len(states),
                     np.int(np.array(temp_states.iloc[:,3:9]).shape[0]/len(states)),6
                                                      ).astype(float),
    'd_precip':np.array(precip_states.iloc[:,3:9]).reshape(
                     len(states),
                     np.int(np.array(precip_states.iloc[:,3:9]).shape[0]/len(states)),6
                                                      ).astype(float),
    'd_yields':np.array(yields[yields["Region"].isin(
                     ['Maize_Spring_USA_'+s for s in states]
                                                    )].iloc[:,22:]).astype(float)+9.75,
    'n_gf':40,
    'temp':np.arange(0,40,1),
    'precip':np.arange(0,200,5)

}

In [ ]:

gm2 = pystan.StanModel(file='./stan/2d-gaussian.stan')

In [ ]:

fit=gm2.sampling(data=data2,chains=4,iter=1000,verbose=True,seed=1308)

In [ ]:

fit

In [ ]:

# carry out some diagnostic checks on fit

stan_utility.check_div(fit)
stan_utility.check_energy(fit)
stan_utility.check_treedepth(fit)

In [ ]:
samples=fit.extract()

In [ ]:
#plt.figure(figsize=(10,10))
for i in range(0,2000,10):
    plt.plot(fit.data['temp'],samples['fdy1'][i,:],alpha=0.1, c='b')
plt.xlabel('Temperature [$^\circ$C]')
plt.ylabel(r'$dy$ [tonnes ha$^{-1}$]')

plt.savefig('./figs/2d_Gauss_temp_post_sample_growth_curve.pdf')
plt.savefig('./figs/2d_Gauss_temp_post_sample_growth_curve.png')
#plt.title('Growth Curve')

In [ ]:
for i in range(0,2000,10):
    plt.plot(fit.data['precip'],samples['fdy2'][i,:],alpha=0.1, c='b')
plt.xlabel('Precipitation [mm]')
plt.ylabel(r'$dy$ [tonnes ha$^{-1}$]')
plt.savefig('./figs/2d_Gauss_precip_post_sample_growth_curve.pdf')
plt.savefig('./figs/2d_Gauss_precip_post_sample_growth_curve.png')

## Compute a sample of region averaged yield anomalies for each posterior sample

We have a sample from the posterior on the Gaussian parameters $\boldsymbol \theta_i$. For each sample, $i$, we want to compute the mean yield anomaly for all the regions combined at a given temperature increment $\Delta T_j$ applied to every tru temperature. After this procedure we will have a sample of mean yield anomalies $Y_i_j$ for each temperature increase.

In [ ]:
def yield_anomaly(temp_6m, precip_6m, mu_t, sigma_t, mu_p, sigma_p, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-0.5 *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                             + np.square((precip_6m[month] - mu_p)/sigma_p) )
                                      )
    return np.sum(dy)


def compute_mean_yield_anomaly(T_inc, data, mu_t, sigma_t, mu_p, sigma_p, norm):
    """ Compute mean yield anomaly for a model over regions and years
    
    The function yield_anomaly returens the yield anomaly for a year 
    and and region. Here we loop over the regions and years to create
    and overall mean.
    
    """
    
    yield_anomalies = np.full((6, 35), np.nan)
    #loop over states
    for state in np.arange(6):
        #loop over years
        for year in np.arange(35):
            temp_6m = data['d_temp'][state, year, :] + T_inc
            precip_6m = data['d_precip'][state, year, :]
            yield_anomalies[state, year] = yield_anomaly(temp_6m, precip_6m, mu_t, sigma_t, mu_p, sigma_p, norm)
    
    return np.nanmean(yield_anomalies)


compute_mean_yield_anomaly(1., data2, 19.71, 5.23, 114.69, 62.41, [1.56])    

In [ ]:
%%timeit
compute_mean_yield_anomaly(1., data2, 20., 5., 120., 65., [1.5]) 

In [ ]:
(200000* 14.e-3)/(60)

There are six regions each with 35 years with six months. We want the total average for each parameter set and temperature increase.

In [ ]:
T_incs = np.linspace(-5, 5, 50)

In [ ]:
np.zeros((len(samples['mu_t']), len(T_incs))).shape

In [ ]:
mean_yield_samples =  np.zeros((len(samples['mu_t']), len(T_incs)))

#Loop over the samples and for each parameter realisation
#TODO: Vectorise to speed up

for n in np.arange(len(samples['mu_t'])):
    
    mu_t    = samples['mu_t'][n]
    sigma_t = samples['sigma_t'][n]
    mu_p    = samples['mu_p'][n]
    sigma_p = samples['sigma_p'][n]
    norm    = [samples['norm'][n]]
    for m, T_inc in enumerate(T_incs):
        mean_yield_samples[n, m] = compute_mean_yield_anomaly(T_inc, data2, mu_t, sigma_t, mu_p, sigma_p, norm)

In [ ]:
len(mean_yield_samples.T[0])

In [ ]:
sample_means   = [np.mean(s) for s in mean_yield_samples.T]
sample_medians = [np.median(s) for s in mean_yield_samples.T]
sample_25      = [np.percentile(s, 25.) for s in mean_yield_samples.T]
sample_75      = [np.percentile(s, 75.)  for s in mean_yield_samples.T]

In [ ]:

plt.plot(T_incs, sample_means, label='means')
#plt.plot(T_incs, sample_medians, label='medians')
#plt.plot(T_incs, sample_25, label='25th percentile')
#plt.plot(T_incs, sample_75, label='75th percentile')
plt.fill_between(T_incs, sample_25, sample_75, label='75th - 25th percentile', alpha=0.2)
plt.legend()
plt.xlabel('T_inc (K)')
plt.ylabel('Yield [tonnes ha$^{-1}$]')
plt.savefig('./figs/temperature_impact.pdf')
plt.savefig('./figs/temperature_impact.png')

In [ ]:
plt.hist(mean_yield_samples.T[33], 20)

In [ ]:
plt.hist(mean_yield_samples.T[0], 20)

This clearly shows that the model predicts reductions in yields as a consequence of raising temperatures.

## Make a two dimensional plot of the function on one posterior

We want to see how the function 

In [ ]:
# Posterior sample
#                    mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
#mu_t               19.71    0.02   0.68  18.17  19.29  19.77  20.19  20.87   1205    1.0
#sigma_t             5.23    0.02   0.58   4.26    4.8   5.17    5.6   6.49    795    1.0
#mu_p              114.69    0.11   4.26 106.07 111.81 114.87 117.65 122.71   1571    1.0
#sigma_p            62.41    0.07   2.76  57.16  60.51  62.41  64.24   67.9   1633    1.0
#norm                1.56  2.3e-3   0.07   1.43   1.51   1.56    1.6    1.7    862    1.0

In [ ]:


fig, ax = plt.subplots()

t1, t2 = 5., 35.
p1, p2 = 0., 250.
x = arange(t1,t2,.1)
y = arange(p1,p2,1.)
X,Y = meshgrid(x, y) # grid of point
def yield_equal(temp, precip, mu_t, sigma_t, mu_p, sigma_p, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    temp_6m = np.full(6, temp)
    precip_6m = np.full(6, precip)
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-0.5 *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                             + np.square((precip_6m[month] - mu_p)/sigma_p) )
                                      )
    return np.sum(dy)
def z_func(X,Y):
    return yield_equal(X, Y, 19.77, 5.17, 114.87, 62.41, [1.56])
z_func = np.vectorize(z_func)
#Lets use the median

Z = z_func(X, Y) # evaluation of the function on the grid

im = imshow(Z,cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

ax.set_xlabel('T [$^{\circ}$C]')
ax.set_ylabel('Monthly precipitation (mm)')

plt.savefig('./figs/2d_yield_response.pdf')
plt.savefig('./figs/2d_yield_response.png')

In [ ]:

fig, ax = plt.subplots()

t1, t2 = 5., 35.
p1, p2 = 0., 250.
x = arange(t1,t2,.1)
y = arange(p1,p2,1.)
X,Y = meshgrid(x, y) # grid of point
def yield_equal(temp, precip, mu_t, sigma_t, mu_p, sigma_p, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    temp_6m = np.full(6, temp)
    precip_6m = np.full(6, precip)
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-0.5 *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                             + np.square((precip_6m[month] - mu_p)/sigma_p) )
                                      )
    return np.sum(dy)
def z_func(X,Y):
    return yield_equal(X, Y, 19.77, 5.17, 114.87, 62.41, [1.56])
z_func = np.vectorize(z_func)
#Lets use the median

Z = z_func(X, Y) # evaluation of the function on the grid

im = imshow(Z,cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(X, Y, Z)
ax.clabel(CS, inline=1, fontsize=10)


ax.set_xlabel('T [$^{\circ}$C]')
ax.set_ylabel('Monthly precipitation (mm)')

plt.savefig('./figs/2d_yield_response_contours.pdf')
plt.savefig('./figs/2d_yield_response_contours.png')

## Include correlation

We originally ran a zero correlated gaussian, now let's incldue correlation


In [ ]:
gm3 = pystan.StanModel(file='./stan/2d-gaussian_with_correlation.stan')

In [ ]:
fit2=gm3.sampling(data=data2,chains=4,iter=1000,verbose=True,seed=1308)

In [ ]:
# carry out some diagnostic checks on fit

stan_utility.check_div(fit2)
stan_utility.check_energy(fit2)
stan_utility.check_treedepth(fit2)

In [ ]:
samples2=fit2.extract()

In [ ]:
fit2

In [ ]:
#plt.figure(figsize=(10,10))
for i in range(0,2000,10):
    plt.plot(fit2.data['temp'],samples['fdy1'][i,:],alpha=0.1, c='b')
plt.xlabel('Temperature [$^{\circ}$C]')
plt.ylabel(r'$dy$')
#plt.title('Growth Curve')
plt.savefig('./figs/growth_curve.pdf')
plt.savefig('./figs/growth_curve.png')

In [ ]:
def yield_anomaly2(temp_6m, precip_6m, mu_t, sigma_t, mu_p, sigma_p, rho, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-(1/(2 - 2*rho**2)) *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                                           + np.square((precip_6m[month] - mu_p)/sigma_p) 
                                                           - (2*rho*(temp_6m[month]    - mu_t)
                                                                   *(precip_6m[month] - mu_p)
                                                             )
                                                              /(sigma_t*sigma_p)
                                                           )
                                      )
                                      
    return np.sum(dy)


def compute_mean_yield_anomaly2(T_inc, data, mu_t, sigma_t, mu_p, sigma_p, rho, norm):
    """ Compute mean yield anomaly for a model over regions and years
    
    The function yield_anomaly returens the yield anomaly for a year 
    and and region. Here we loop over the regions and years to create
    and overall mean.
    
    """
    
    yield_anomalies = np.full((6, 35), np.nan)
    #loop over states
    for state in np.arange(6):
        #loop over years
        for year in np.arange(35):
            temp_6m = data['d_temp'][state, year, :] + T_inc
            precip_6m = data['d_precip'][state, year, :]
            yield_anomalies[state, year] = yield_anomaly2(temp_6m, 
                                                         precip_6m, 
                                                         mu_t, 
                                                         sigma_t, 
                                                         mu_p, 
                                                         sigma_p,
                                                         rho, 
                                                         norm)
    
    return np.nanmean(yield_anomalies)


compute_mean_yield_anomaly2(1., data2, 19.71, 5.23, 114.69, 62.41, 0.0, [1.56])  

In [ ]:


#Loop over the samples and for each parameter realisation

T_incs_corr = np.linspace(-5, 5, 50)
mean_yield_samples2 =  np.full((len(samples2['mu_t']), len(T_incs_corr)), np.nan)

for n in np.arange(len(samples2['mu_t'])):
    if n % 100 == 0:
        print("{} out of {}".format(n, len(samples2['mu_t'])))
    mu_t    = samples2['mu_t'][n]
    sigma_t = samples2['sigma_t'][n]
    mu_p    = samples2['mu_p'][n]
    sigma_p = samples2['sigma_p'][n]
    rho = samples2['rho'][n]
    norm    = [samples2['norm'][n]]
    for m, T_inc in enumerate(T_incs_corr):
        mean_yield_samples2[n, m] = compute_mean_yield_anomaly2(T_inc, data2, mu_t, sigma_t, mu_p, sigma_p, rho, norm)

In [ ]:
np.save('mean_yield_samples2', mean_yield_samples2)

In [ ]:
sample2_means   = [np.mean(s) for s in mean_yield_samples2.T]
sample2_medians = [np.median(s) for s in mean_yield_samples2.T]
sample2_25      = [np.percentile(s, 25.) for s in mean_yield_samples2.T]
sample2_75      = [np.percentile(s, 75.)  for s in mean_yield_samples2.T]

In [ ]:

plt.plot(T_incs_corr, sample2_means, label='means')
#plt.plot(T_incs, sample_medians, label='medians')
#plt.plot(T_incs, sample_25, label='25th percentile')
#plt.plot(T_incs, sample_75, label='75th percentile')
plt.fill_between(T_incs_corr, sample2_25, sample2_75, label='75th - 25th percentile', alpha=0.2)
plt.legend()
plt.xlabel('T$_{{inc}}$ (K)')
plt.ylabel('Yield [tonnes ha$^{-1}$]')
plt.savefig('./figs/temperature_impact2_m5p5.pdf')
plt.savefig('./figs/temperature_impact2_m5p5.png')

In [ ]:
#                    mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
#mu_t               19.67    0.02   0.72  18.09  19.24  19.74  20.18  20.93   1036    1.0
#sigma_t             5.23    0.02   0.57   4.32   4.83   5.17   5.57   6.53    797    1.0
#mu_p              115.07    0.11   4.12 106.99 112.34 115.12 117.78 123.13   1443    1.0
#sigma_p            62.62    0.08   2.82  57.36  60.63  62.57   64.5  68.19   1358    1.0
#rho                -0.02  2.0e-3   0.08  -0.17  -0.07  -0.02   0.03   0.13   1433    1.0
#norm                1.56  2.1e-3   0.07   1.44   1.51   1.56    1.6   1.69    973    1.0

In [ ]:


fig, ax = plt.subplots()

t1, t2 = 5., 35.
p1, p2 = 0., 250.
x = np.linspace(t1,t2,100.)
y = np.linspace(p1,p2,100.)
X,Y = meshgrid(x, y) # grid of point
def yield_equal2(temp, precip, mu_t, sigma_t, mu_p, sigma_p, rho, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    temp_6m = np.full(6, temp)
    precip_6m = np.full(6, precip)
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-(1/(2 - 2*rho**2)) *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                                           + np.square((precip_6m[month] - mu_p)/sigma_p) 
                                                           - (2*rho*(temp_6m[month]    - mu_t)
                                                                   *(precip_6m[month] - mu_p)
                                                             )
                                                              /(sigma_t*sigma_p)
                                                           )
                                      )
    return np.sum(dy)
def z_func(X,Y):
    return yield_equal2(X, Y, 19.74, 5.17, 115.12, 62.57, -0.02, [1.56])
z_func = np.vectorize(z_func)
#Lets use the median

Z = z_func(X, Y) # evaluation of the function on the grid

im = ax.imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)

cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

ax.set_xlabel('T (Celsius)')
ax.set_ylabel('Monthly precipitation (mm)')



plt.savefig('./figs/2d_2.pdf')
plt.savefig('./figs/2d_2.png')

In [ ]:
fig, ax = plt.subplots()

t1, t2 = 5., 35.
p1, p2 = 0., 250.
x = np.linspace(t1,t2,100.)
y = np.linspace(p1,p2,100.)
X,Y = meshgrid(x, y) # grid of point
def yield_equal2(temp, precip, mu_t, sigma_t, mu_p, sigma_p, rho, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    temp_6m = np.full(6, temp)
    precip_6m = np.full(6, precip)
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-(1/(2 - 2*rho**2)) *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                                           + np.square((precip_6m[month] - mu_p)/sigma_p) 
                                                           - (2*rho*(temp_6m[month]    - mu_t)
                                                                   *(precip_6m[month] - mu_p)
                                                             )
                                                              /(sigma_t*sigma_p)
                                                           )
                                      )
    return np.sum(dy)
def z_func(X,Y):
    return yield_equal2(X, Y, 19.74, 5.17, 115.12, 62.57, -0.02, [1.56])
z_func = np.vectorize(z_func)
#Lets use the median

Z = z_func(X, Y) # evaluation of the function on the grid

im = ax.imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)

cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

ax.set_xlabel('T (Celsius)')
ax.set_ylabel('Monthly precipitation (mm)')

CS = ax.contour(X, Y, Z, 3, colors='w')
ax.clabel(CS, inline=1, fmt='%1.1f', fontsize=10)

plt.savefig('./figs/2d_2_contour.pdf')
plt.savefig('./figs/2d_2_contour.png')

In [ ]:
data2['d_precip']

## Now lets plot a response in temperature and precipitation space

In [ ]:
T_incs_2 = np.linspace(-10, 10, 50)
P_incs_2 = np.linspace(-100, 100, 50)
T_P_grid = np.meshgrid(T_incs_2, P_incs_2)

In [ ]:
def yield_anomaly2(temp_6m, precip_6m, mu_t, sigma_t, mu_p, sigma_p, rho, norm):
    """Take six months of T and P and return yield for given params.
    
    This should be identical to the function in the STAN model
    """
    if len(norm) == 1:
        norm = norm * np.ones(6)
    dy = np.zeros(6)
    for month in np.arange(6):
        dy[month] = norm[month]*np.exp(-(1/(2 - 2*rho*2)) *( np.square((temp_6m[month]    - mu_t)/sigma_t) 
                                                           + np.square((precip_6m[month] - mu_p)/sigma_p) 
                                                           - (2*rho*(temp_6m[month]    - mu_t)
                                                                   *(precip_6m[month] - mu_p)
                                                             )
                                                              /(sigma_t*sigma_p)
                                                           )
                                      )
                                      
    return np.sum(dy)

def compute_mean_yield_anomaly3(T_inc, P_inc, data, mu_t, sigma_t, mu_p, sigma_p, rho, norm):
    """ Compute mean yield anomaly for a model over regions and years
    
    The function yield_anomaly returens the yield anomaly for a year 
    and and region. Here we loop over the regions and years to create
    and overall mean.
    
    """
    
    yield_anomalies = np.full((6, 35), np.nan)
    #loop over states
    for state in np.arange(6):
        #loop over years
        for year in np.arange(35):
            temp_6m = data['d_temp'][state, year, :] + T_inc
            precip_6m = data['d_precip'][state, year, :] + P_inc
            yield_anomalies[state, year] = yield_anomaly2(temp_6m, 
                                                         precip_6m, 
                                                         mu_t, 
                                                         sigma_t, 
                                                         mu_p, 
                                                         sigma_p,
                                                         rho, 
                                                         norm)
    
    return np.nanmean(yield_anomalies)

In [ ]:
take_100 = np.random.choice([0,1], 
                            size=len(samples2['mu_t']), 
                            p=(1-100/len(samples2['mu_t']), 100/len(samples2['mu_t'])))


In [ ]:
mean_yield_anom = np.full((len(T_incs_2), len(P_incs_2)), np.nan)

print("Starting at {}".format(time.time()))
for n, t in enumerate(T_incs_2):
    print("{} out of {}".format(n, len(T_incs_2)))
    for m, p in enumerate(P_incs_2):
        mean_yield_samples = np.full(len(samples2['mu_t']), np.nan)
        for k in np.arange(len(samples2['mu_t'])):
            if not take_100[k]:
                continue
            
            #print('k = {}'.format(k))
            mu_t    = samples2['mu_t'][k]
            sigma_t = samples2['sigma_t'][k]
            mu_p    = samples2['mu_p'][k]
            sigma_p = samples2['sigma_p'][k]
            rho = samples2['rho'][k]
            norm    = [samples2['norm'][k]]

            mean_yield_samples[k] = compute_mean_yield_anomaly3(t, p, data2, mu_t, sigma_t, mu_p, sigma_p, rho, norm)
        
        mean_yield_anom[n, m] = np.nanmean(mean_yield_samples)

In [ ]:
np.save('mean_yield_anom', mean_yield_anom)

In [ ]:



fig, ax = plt.subplots()


im = imshow(np.flip(mean_yield_anom.T, axis=0),cmap="viridis", 
            extent=[T_incs_2[0],T_incs_2[-1], P_incs_2[0], P_incs_2[-1]], 
            aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

ax.set_xlabel('$T_{inc}$ [K]')
ax.set_ylabel('$P_{inc}$ [mm]')

plt.savefig('./figs/temp_precip_impact.pdf')
plt.savefig('./figs/temp_precip_impact.png')


In [ ]:
np.max(mean_yield_anom.T)

In [ ]:
mean_yield_anom.T[int(len(T_incs_2)/2), int(len(P_incs_2)/2)]

In [ ]:
0.9 * mean_yield_anom.T[int(len(T_incs_2)/2), int(len(P_incs_2)/2)]

In [ ]:

fig, ax = plt.subplots()


im = imshow(np.flip(mean_yield_anom.T, axis=0),cmap="viridis", 
            extent=[T_incs_2[0],T_incs_2[-1], P_incs_2[0], P_incs_2[-1]], 
            aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
cbar = fig.colorbar(im) # adding the colobar on the right
cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

ax.set_xlabel('$T_{inc}$ [K]')
ax.set_ylabel('$P_{inc}$ [mm]')

CS = ax.contour(T_incs_2, P_incs_2, mean_yield_anom.T, [4., 
               6., 
               8., 
               0.9 * mean_yield_anom.T[int(len(T_incs_2)/2), int(len(P_incs_2)/2)]
              ], colors='w')

fmt = {}
strs = ['4', '6', '8', '-10% (8.8)']
for l, s in zip(CS.levels, strs):
    fmt[l] = s
ax.clabel(CS, 
          inline=1, 
          fmt=fmt, 
          fontsize=10)

plt.savefig('./figs/temp_precip_impact_contour.pdf')
plt.savefig('./figs/temp_precip_impact_contour.png')

## Plot posteriors

In [ ]:
samples = samples2

In [ ]:
df=pd.DataFrame(np.random.multivariate_normal(np.array([20,5]),np.array([[5.0,0.0],[0.0,1.0]]),2000),
                columns=['$\mu_T$','$\sigma_T$'])
g=sns.PairGrid(data=df,size=2.5,diag_sharey=False)
g.map_diag(plt.hist,color='Red',alpha=0.5)
g.map_lower(sns.kdeplot, cmap="Reds",alpha=0.8,n_levels=10,normed=True, shade=True,shade_lowest=False)
df=pd.DataFrame(np.vstack((samples['mu_t'],samples['sigma_t'])).T,columns=['$\mu_T$','$\sigma_T$'])
g.data=df
g.map_diag(plt.hist,color='Blue',alpha=0.5)
g.map_lower(sns.kdeplot, cmap="Blues",alpha=0.8,n_levels=10,normed=True, shade=True,shade_lowest=False)

g.axes[0,1].set_axis_off()

#fig = g.get_figure()
plt.savefig("./figs/2d_Gauss_prior_vs_post_temp_mean_vs_sigma.png")
plt.savefig("./figs/2d_Gauss_prior_vs_post_temp_mean_vs_sigma.pdf")

In [ ]:
df=pd.DataFrame(np.random.multivariate_normal(np.array([100,25]),np.array([[25.0,0.0],[0.0,5.0]]),2000),
                columns=['$\mu_p$','$\sigma_p$'])
g=sns.PairGrid(data=df,size=2.5,diag_sharey=False)
g.map_diag(plt.hist,color='Red',alpha=0.5)
g.map_lower(sns.kdeplot, cmap="Reds",alpha=0.8,n_levels=10,normed=True, shade=True,shade_lowest=False)
df=pd.DataFrame(np.vstack((samples['mu_p'],samples['sigma_p'])).T,columns=['$\mu_p$','$\sigma_p$'])

g.data=df
g.map_diag(plt.hist,color='Blue',alpha=0.5)
g.map_lower(sns.kdeplot, cmap="Blues",alpha=0.8,n_levels=10,normed=True, shade=True,shade_lowest=False)

g.axes[0,1].set_axis_off()

#fig = g.get_figure()
plt.savefig("./figs/2d_Gauss_prior_vs_post_precip_mean_vs_sigma.png")
plt.savefig("./figs/2d_Gauss_prior_vs_post_precip_mean_vs_sigma.pdf")

## Pvals

In [ ]:
def Bayesian_Pval(yields,pred_yields):
    import scipy.stats as st
    Pvals=np.empty_like(yields)
    n_reg,n_years=yields.shape
    for r in range(0,n_reg):
        for y in range(0,n_years):
            ind=pred_yields[:,r,y]<yields[r,y]
            Pvals[r,y]=st.norm.ppf(ind.sum()/pred_yields[:,r,y].size)
    return Pvals

In [ ]:
Pvals=Bayesian_Pval(fit2.data['d_yields'],samples2['pred_yields'])

In [ ]:
#plt.figure(figsize=(10,5))
for s in range(0,len(states)):
    plt.plot(Pvals[s,:],label=states[s])
plt.xticks(5*np.arange(len(np.arange(1980,2015, 5))),np.arange(1980,2015, 5),rotation=90);
plt.legend()
plt.ylabel('P value')

plt.savefig("./figs/pvalues_all.png".format(states[s]))
plt.savefig("./figs/pvalues_all.pdf".format(states[s]))